TASK 1

In [12]:
import keras
from tensorflow import keras
import nltk
import pickle
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
import random
import datetime
from googlesearch import search
import webbrowser
import requests
from pycricbuzz import Cricbuzz
import billboard
import time
from pygame import mixer
from nltk.stem import WordNetLemmatizer
from tkinter import *
from keras.models import load_model

In [8]:
lemmatizer=WordNetLemmatizer()

word_dict=[]
classes=[]
docs=[]
ignore=['?','!',',',"'s"]

data_file=open('intents.json').read()
intents=json.loads(data_file)

for i in intents['intents']:
    for p in i['patterns']:
        w=nltk.word_tokenize(p)
        word_dict.extend(w)
        docs.append((w,i['tag']))
        
        if i['tag'] not in classes:
            classes.append(i['tag'])
            
word_dict=[lemmatizer.lemmatize(word.lower()) for word in word_dict if word not in ignore]
word_dict=sorted(list(set(word_dict)))
classes=sorted(list(set(classes)))
pickle.dump(word_dict,open('word_dict.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [10]:
train_list=[]
output_empty=[0 for _ in range(len(classes))]

for doc in docs:
    bag=[]
    pattern=doc[0]
    pattern=[ lemmatizer.lemmatize(word.lower()) for word in pattern ]
    
    for word in word_dict:
        if word in pattern:
            bag.append(1)
        else:
            bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    train_list.append([bag,output_row])
    
random.shuffle(train_list)
train_list=np.array(train_list)  
x_train=list(train_list[:,0])
y_train=list(train_list[:,1])  


C:\Users\Sachi\AppData\Local\Temp\ipykernel_17340\1705832986.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_list=np.array(train_list)


In [11]:
model=Sequential()
model.add(Dense(128,activation='relu',input_shape=(len(x_train[0]),)))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(y_train[0]),activation='softmax'))

adam= keras.optimizers.Adam(0.001)

model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
weights=model.fit(np.array(x_train),np.array(y_train),epochs=350,batch_size=5)    
model.save('mymodel.h5',weights)

Epoch 1/350
21/21 [==============================] - 1s 2ms/step - loss: 3.2842 - accuracy: 0.0583 
Epoch 2/350
21/21 [==============================] - 0s 2ms/step - loss: 3.2403 - accuracy: 0.0680
Epoch 3/350
21/21 [==============================] - 0s 2ms/step - loss: 3.2055 - accuracy: 0.0874
Epoch 4/350
21/21 [==============================] - 0s 2ms/step - loss: 3.1602 - accuracy: 0.1262
Epoch 5/350
21/21 [==============================] - 0s 2ms/step - loss: 3.0776 - accuracy: 0.1553
Epoch 6/350
21/21 [==============================] - 0s 3ms/step - loss: 3.0164 - accuracy: 0.1845
Epoch 7/350
21/21 [==============================] - 0s 6ms/step - loss: 2.8801 - accuracy: 0.1942
Epoch 8/350
21/21 [==============================] - 0s 6ms/step - loss: 2.7795 - accuracy: 0.2233
Epoch 9/350
21/21 [==============================] - 0s 5ms/step - loss: 2.6379 - accuracy: 0.3204
Epoch 10/350
21/21 [==============================] - 0s 5ms/step - loss: 2.4500 - accuracy: 0.3301
Epoch 11

In [13]:
model = load_model('mymodel.h5')

intents = json.loads(open('intents.json').read())
word_dict = pickle.load(open('word_dict.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [17]:
def lemmatization(sent):
    sent_words=nltk.word_tokenize(sent)
    sent_words=[ lemmatizer.lemmatize(w.lower()) for w in sent_words]
    return sent_words

def create_bag(sent,word_dict):
    sent_words=lemmatization(sent)
    temp_bag=list(np.zeros(len(word_dict)))
    
    for s in sent_words:
        for i,w in enumerate(word_dict):
            if w == s: 
                temp_bag[i] = 1
    return np.array(temp_bag)

def prediction(sent,model):
    p=create_bag(sent,word_dict)
    res=model.predict(np.array([p]))[0]
    threshold=0.8
    results=[[i,r] for i,r in enumerate(res) if r>threshold]
    results.sort(key=lambda x: x[1],reverse=True)
    return_list=[]
    
    for result in results:
        return_list.append({'intent':classes[result[0]],'prob':str(result[1])})
    return return_list

def get_response(return_list,intents_json):
    
    if len(return_list)==0:
        tag='noanswer'
    else:    
        tag=return_list[0]['intent']
    if tag=='datetime':        
        print(time.strftime("%A"))
        print (time.strftime("%d %B %Y"))
        print (time.strftime("%H:%M:%S"))

    if tag=='google':
        query=input('Enter query...')
        chrome_path = r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe %s'
        for url in search(query, tld="co.in", num_results=1, stop = 1, pause = 2):
            webbrowser.open("https://google.com/search?q=%s" % query)
        
    if tag=='news':
        main_url = " http://newsapi.org/v2/top-headlines?country=in&apiKey=bc88c2e1ddd440d1be2cb0788d027ae2"
        open_news_page = requests.get(main_url).json()
        article = open_news_page["articles"]
        results = [] 
          
        for ar in article: 
            results.append([ar["title"],ar["url"]]) 
          
        for i in range(10): 
            print(i + 1, results[i][0])
            print(results[i][1],'\n')
            
    if tag=='cricket':
        c = Cricbuzz()
        matches = c.matches()
        for match in matches:
            print(match['srs'],' ',match['mnum'],' ',match['status'])
    
    if tag=='song':
        chart=billboard.ChartData('hot-100')
        print('The top 10 songs at the moment are:')
        for i in range(10):
            song=chart[i]
            print(song.title,'- ',song.artist)
    if tag=='timer':        
        mixer.init()
        x=input('Minutes to timer..')
        time.sleep(float(x)*60)

    list_of_intents= intents_json['intents'] 
    result = str()   
    for i in list_of_intents:
        if tag==i['tag'] :
            result= random.choice(i['responses'])
    return result

def response(text):
    return_list=predict_class(text,model)
    response=get_response(return_list,intents)
    return response

while(1):
    x=input()
    print(x)
    print(response(x))
    if x.lower() in ['bye','goodbye','get lost','see you']:  
        break



hi
1/1 [==============================] - 0s 32ms/step
Hi there, how can I help?
google
1/1 [==============================] - 0s 32ms/step
Redirecting to Google...
bye
1/1 [==============================] - 0s 33ms/step
See you!
